# **Compare models [High-Speed face Emotion recognition](https://github.com/HSE-asavchenko/face-emotion-recognition) vs [Residual Masking Network](https://github.com/phamquiluan/ResidualMaskingNetwork)**

In [1]:
!pip install hsemotion==0.3.0 opencv-python==4.8.0.74 pillow rmn==3.1.1 timm==0.6.5

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.8/512.8 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.4/532.4 kB 48.6 MB/s eta 0:00:00
  Created wheel for hsemotion: filename=hsemotion-0.3.0-py3-none-any.whl size=11242 sha256=1a50af70e611e9a3d9bbf775a1c60866042ea85a8ca4055d2fa486318be37a76
  Stored in directory: /root/.cache/pip/wheels/38/88/e0/3b365122443c2ec55f3e058f2b7ad59df7b5e302c457c4539a
Successfully built hsemotion
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.7.0.72
    Uninstalling opencv-python-4.7.0.72:
      Successfully uninstalled opencv-python-4.7.0.72


In [2]:
!curl -L -o affectnet.zip https://www.dropbox.com/s/miltxghiqvnu98d/AffectNet.zip?dl=0
!7z x affectnet.zip -oaffectnet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    17    0    17    0     0     54      0 --:--:-- --:--:-- --:--:--    54
100   340  100   340    0     0    503      0 --:--:-- --:--:-- --:--:--   503
100   534    0   534    0     0    556      0 --:--:-- --:--:-- --:--:--  9709
100  314M  100  314M    0     0  66.7M      0  0:00:04  0:00:04 --:--:--  109M

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 329755345 bytes (315 MiB)

Extracting archive: affectnet.zip
 14% 4096 Open              --
Path = affectnet.zip
Type = zip
Physical Size = 329755345

  0%      0% 521 - anger/image0009907.jpg

In [3]:
import pathlib
import time

import cv2
import numpy
import rmn
import torch
from hsemotion.facial_emotions import HSEmotionRecognizer
from PIL import Image

pretrained_ckpt does not exists!


deploy.prototxt.txt does not exists!


res10_300x300_ssd_iter_140000.caffemodel does not exists!


In [4]:
subdir_to_class = {"anger": 0, "disgust": 1, "fear": 2, "happy": 3, "neutral": 4, "sad": 5, "surprise": 6}
hse_label_to_class = {"Anger": 0, "Disgust": 1, "Fear": 2, "Happiness": 3, "Neutral": 4, "Sadness": 5, "Surprise": 6}
rmn_label_to_class = {"angry": 0, "disgust": 1, "fear": 2, "happy": 3, "neutral": 4, "sad": 5, "surprise": 6}

In [9]:
def test(images, hse_fer, rmn_fer, logging_step=1000):
    hse_num_matches = 0
    rmn_num_matches = 0
    hse_total_duration = 0
    rmn_total_duration = 0
    num_images = 0
    for i, (true_class, img) in enumerate(images):
        num_images += 1
        img = numpy.asarray(img)
        img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        hse_duration, (hse_label, *_) = timed(hse_fer.predict_emotions, img)
        hse_num_matches += hse_label_to_class.get(hse_label) == true_class
        hse_total_duration += hse_duration

        rmn_duration, (rmn_label, *_) = timed(rmn_fer.detect_emotion_for_single_face_image, img_bgr)
        rmn_num_matches += rmn_label_to_class[rmn_label] == true_class
        rmn_total_duration += rmn_duration

        if i % logging_step == 0:
            print(
                "Accuracy: HSE={0:.3f} RMN={1:.3f} | Duration: HSE={2:.3f} RMN={3:.3f}".format(
                    hse_num_matches / num_images,
                    rmn_num_matches / num_images,
                    hse_total_duration / num_images,
                    rmn_total_duration / num_images,
                )
            )

    print("Images:", num_images)
    print(
        "Accuracy: HSE={0:.3f} RMN={1:.3f} | Duration: HSE={2:.3f} RMN={3:.3f}".format(
            hse_num_matches / num_images,
            rmn_num_matches / num_images,
            hse_total_duration / num_images,
            rmn_total_duration / num_images,
        )
    )

def iter_images(root):
    class_to_images = {class_: (pathlib.Path(root) / subdir).iterdir() for subdir, class_ in subdir_to_class.items()}
    while class_to_images:
        for class_, images in list(class_to_images.items()):
            try:
                yield class_, Image.open(next(images))
            except StopIteration:
                class_to_images.pop(class_)


def timed(fn, *args):
    started = time.monotonic()
    result = fn(*args)
    finished = time.monotonic()
    return finished - started, result

**Let's compare the models in accuracy:**

In [10]:
test(
    iter_images("affectnet"),
    HSEmotionRecognizer(device="cuda" if torch.cuda.is_available() else "cpu"),
    rmn.RMN(),
)

/root/.hsemotion/enet_b0_8_best_vgaf.pt Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)
Accuracy: HSE=1.000 RMN=1.000 | Duration: HSE=0.014 RMN=0.034
Accuracy: HSE=0.638 RMN=0.494 | Duration: HSE=0.022 RMN=0.023
Accuracy: HSE=0.656 RMN=0.496 | Duration: HSE=0.021 RMN=0.022
Accuracy: HSE=0.648 RMN=0.493 | Duration: HSE=0.021 RMN=0.022
Accuracy: HSE=0.650 RMN=0.493 | Duration: HSE=0.021 RMN=0.022
Accuracy: HSE=0.646 RMN=0.490 | Duration: HSE=0.021 RMN=0.022
Accuracy: HSE=0.646 RMN=0.486 | Duration: HSE=0.021 RMN=0.022
Accuracy: HSE=0.649 RMN=0.489 | Duration: HSE=0.021 RMN=0.022
Accuracy: HSE=0.649 RMN=0.491 | Duration: HSE=0.021 RMN=0.022
Accuracy: HSE=0.652 RMN=0.490 | Duration: HSE=0.021 RMN=0.022
Accuracy: HSE=0.651 RMN=0.489 | Duration: HSE=0.021 RMN=0.022
Accuracy: HSE=0.652 RMN=0.489 | Duration: HSE=0.021 RMN=0.022
Accuracy: HSE=0.651 RMN=0.486 | Durati

**The model will be used on cpu only. Therefore let's estimate perfomance of models on CPU:**

In [11]:
import itertools

rmn.is_cuda = False
test(
    itertools.islice(iter_images("affectnet"), 100),
    HSEmotionRecognizer(device="cpu"),
    rmn.RMN(),
    logging_step=10,
)

/root/.hsemotion/enet_b0_8_best_vgaf.pt Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)
Accuracy: HSE=1.000 RMN=1.000 | Duration: HSE=0.195 RMN=0.926
Accuracy: HSE=0.818 RMN=0.545 | Duration: HSE=0.071 RMN=0.977
Accuracy: HSE=0.810 RMN=0.524 | Duration: HSE=0.064 RMN=0.990
Accuracy: HSE=0.871 RMN=0.516 | Duration: HSE=0.062 RMN=0.954
Accuracy: HSE=0.805 RMN=0.537 | Duration: HSE=0.061 RMN=0.964
Accuracy: HSE=0.745 RMN=0.510 | Duration: HSE=0.061 RMN=0.969
Accuracy: HSE=0.705 RMN=0.475 | Duration: HSE=0.061 RMN=0.970
Accuracy: HSE=0.676 RMN=0.479 | Duration: HSE=0.060 RMN=0.960
Accuracy: HSE=0.704 RMN=0.481 | Duration: HSE=0.060 RMN=0.959
Accuracy: HSE=0.681 RMN=0.473 | Duration: HSE=0.060 RMN=0.962
Images: 100
Accuracy: HSE=0.660 RMN=0.470 | Duration: HSE=0.060 RMN=0.965


**HSE model is better both in accuracy and perfomance**